In [ ]:
!pip uninstall -y kaggle
!pip install kaggle==1.5.6
%env KAGGLE_USERNAME = olaadel
%env KAGGLE_KEY = 66fb44d480b787b525f6ac8060e8f5b0
!kaggle competitions download -c fcis-sc-deeplearning-competition
!unzip /content/fcis-sc-deeplearning-competition.zip

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import keras
import os
import pandas as pd
import tqdm
from keras.preprocessing.image import ImageDataGenerator
from keras import Model, layers
from keras.models import load_model, model_from_json
import tensorflow as tf
from keras.applications.nasnet import NASNetLarge
from keras.applications.nasnet import preprocess_input
from keras.preprocessing import image
import PIL

TEST_DIR = '/content/test/test'
TRAIN_DIR = '/content/train/train'


In [ ]:

train_datagen = ImageDataGenerator(rescale=1./255,
    shear_range=10,
    zoom_range=0.2,
    horizontal_flip=True,
    preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    batch_size=8,
    class_mode='categorical',
    target_size=(331,331))


In [ ]:


conv_base = NASNetLarge(
    include_top=False,
    weights='imagenet')
print('1')
for layer in conv_base.layers:
    layer.trainable = False
x = conv_base.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(128, activation='relu')(x) 
predictions = layers.Dense(10, activation='softmax')(x)
model = Model(conv_base.input, predictions)
print('2')


optimizer = keras.optimizers.Adam()
from keras.optimizers import SGD
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
print('3')

step_size_train = train_generator.n//train_generator.batch_size
print('done')
model.fit_generator(train_generator,
                              samples_per_epoch = step_size_train,  # added in Kaggle
                              epochs = 50,
                              validation_data=0,
                              validation_steps=10  # added in Kaggle
                             )
#model.save('NASnet.h5')




testing_pre = []

for imagep in os.listdir(TEST_DIR):
    image_path = os.path.join(TEST_DIR, imagep)
    img = image.load_img(image_path, target_size=(331,331))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis = 0)
    x = preprocess_input(x)
    preds = model.predict(x)
    print(imagep, preds, np.argmax(preds) + 1)
    testing_pre.append([imagep,np.argmax(preds) + 1])

Csv = pd.DataFrame(testing_pre)
Csv.to_csv("S4.csv")


print('end')

In [ ]:
model.save('/content/drive/My Drive/NNs/base_conv_model11.h5')

In [ ]:
conv_base = NASNetLarge(
    include_top=False,
    weights=None)
print('1')
x = conv_base.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(128, activation='relu')(x) 
predictions = layers.Dense(10, activation='softmax')(x)
model = Model(conv_base.input, predictions)
print('2')
model.load_weights('/content/drive/My Drive/NNs/base_conv_model11.h5')
for layer in model.layers:
  if(layer.name == 'adjust_conv_projection_14'):
    break
  layer.trainable = False
for layer in model.layers:
    print("{0}:\t{1}".format(layer.trainable, layer.name)) 

In [ ]:
optimizer = keras.optimizers.Adam
model.compile(optimizer=optimizer(lr=1e-5), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print('3')

step_size_train = train_generator.n//train_generator.batch_size
train_generator.reset()
print('done')
model.fit_generator(train_generator,
                              samples_per_epoch = step_size_train,  # added in Kaggle
                              epochs = 170,
                              validation_data=0,
                              validation_steps=10  # added in Kaggle
                             )

In [ ]:
testing_pre = []

for imagep in os.listdir(TEST_DIR):
    image_path = os.path.join(TEST_DIR, imagep)
    img = image.load_img(image_path, target_size=(331,331))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis = 0)
    x = preprocess_input(x)
    preds = model.predict(x)
    #print(imagep, preds, np.argmax(preds) + 1)
    testing_pre.append([imagep,np.argmax(preds) + 1])

Csv = pd.DataFrame(testing_pre)
Csv.to_csv("Sfrxxt.csv")

#model.save('/content/drive/My Drive/NNs/base_conv_model11.h5')